## Get the Data

Either use the provided .csv file or (optionally) get fresh (the freshest?) data from running an SQL query on StackExchange: 

Follow this link to run the query from [StackExchange](https://data.stackexchange.com/stackoverflow/query/675441/popular-programming-languages-per-over-time-eversql-com) to get your own .csv file

<code>
select dateadd(month, datediff(month, 0, q.CreationDate), 0) m, TagName, count(*)
from PostTags pt
join Posts q on q.Id=pt.PostId
join Tags t on t.Id=pt.TagId
where TagName in ('java','c','c++','python','c#','javascript','assembly','php','perl','ruby','visual basic','swift','r','object-c','scratch','go','swift','delphi')
and q.CreationDate < dateadd(month, datediff(month, 0, getdate()), 0)
group by dateadd(month, datediff(month, 0, q.CreationDate), 0), TagName
order by dateadd(month, datediff(month, 0, q.CreationDate), 0)
</code>

## Import Statements

In [11]:
import pandas as pd

## Data Exploration

**Challenge**: Read the .csv file and store it in a Pandas dataframe

In [12]:
df = pd.read_csv('QueryResults.csv')
df = df.rename(columns={'m':'DATE', 'TagName':'TAG', 'Unnamed: 2':'POSTS'})

**Challenge**: Examine the first 5 rows and the last 5 rows of the of the dataframe

In [13]:
df.head()

,DATE,TAG,POSTS
0,2008-07-01 00:00:00,c#,3
1,2008-08-01 00:00:00,assembly,8
2,2008-08-01 00:00:00,c,83
3,2008-08-01 00:00:00,c#,506
4,2008-08-01 00:00:00,c++,164


In [14]:
df.tail()

,DATE,TAG,POSTS
2295,2022-05-01 00:00:00,php,4173
2296,2022-05-01 00:00:00,python,23813
2297,2022-05-01 00:00:00,r,4916
2298,2022-05-01 00:00:00,ruby,506
2299,2022-05-01 00:00:00,swift,1804


**Challenge:** Check how many rows and how many columns there are. 
What are the dimensions of the dataframe?

In [15]:
print(f'Rows: {len(df)}')
print(f'Columns: {len(df.columns)}')

Rows: 2300
Columns: 3


**Challenge**: Count the number of entries in each column of the dataframe

In [16]:
for col in df.columns:
    rows = len(df[col].notnull())
    print(f'There are {rows} non-empty columns in {col}')

There are 2300 non-empty columns in DATE
There are 2300 non-empty columns in TAG
There are 2300 non-empty columns in POSTS


**Challenge**: Calculate the total number of post per language.
Which Programming language has had the highest total number of posts of all time?

In [23]:
df_lang = df.groupby('TAG').sum()
df_lang.sort_values('POSTS', ascending=False)

,POSTS
TAG,
javascript,2375506
python,1951262
java,1844695
c#,1538436
php,1434649
c++,765028
r,449639
c,379749
swift,312886


Some languages are older (e.g., C) and other languages are newer (e.g., Swift). The dataset starts in September 2008.

**Challenge**: How many months of data exist per language? Which language had the fewest months with an entry? 


In [34]:
df_date = df.groupby('TAG').count()
df_date = df_date[['DATE']]
df_date.sort_values('DATE', ascending=False)
# df_date

,DATE
TAG,
c#,167
assembly,166
c,166
c++,166
delphi,166
java,166
javascript,166
perl,166
php,166


## Data Cleaning

Let's fix the date format to make it more readable. We need to use Pandas to change format from a string of "2008-07-01 00:00:00" to a datetime object with the format of "2008-07-01"

In [40]:
df2 = df.copy()
df2['DATE'] = pd.to_datetime(df2['DATE'])
df2.head()

,DATE,TAG,POSTS
0,2008-07-01,c#,3
1,2008-08-01,assembly,8
2,2008-08-01,c,83
3,2008-08-01,c#,506
4,2008-08-01,c++,164


## Data Manipulation



**Challenge**: What are the dimensions of our new dataframe? How many rows and columns does it have? Print out the column names and print out the first 5 rows of the dataframe.

**Challenge**: Count the number of entries per programming language. Why might the number of entries be different? 

## Data Visualisaton with with Matplotlib


**Challenge**: Use the [matplotlib documentation](https://matplotlib.org/3.2.1/api/_as_gen/matplotlib.pyplot.plot.html#matplotlib.pyplot.plot) to plot a single programming language (e.g., java) on a chart.

**Challenge**: Show two line (e.g. for Java and Python) on the same chart.

# Smoothing out Time Series Data

Time series data can be quite noisy, with a lot of up and down spikes. To better see a trend we can plot an average of, say 6 or 12 observations. This is called the rolling mean. We calculate the average in a window of time and move it forward by one overservation. Pandas has two handy methods already built in to work this out: [rolling()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rolling.html) and [mean()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.window.rolling.Rolling.mean.html). 